<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/FileCKD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
DrugDF = pd.read_excel('/content/drive/MyDrive/ckdresearch2023/Assign/SummaryFile/WideTable/DrugDF.xlsx')
DrugDF.shape

(52619, 28)

In [3]:
import pandas as pd
DiagDF = pd.read_excel('/content/drive/MyDrive/ckdresearch2023/Assign/SummaryFile/WideTable/DiagDF.xlsx')
DiagDF.shape

(52619, 15)

In [4]:
import pandas as pd
LabDF = pd.read_excel('/content/drive/MyDrive/ckdresearch2023/Assign/SummaryFile/WideTable/LabDF.xlsx')
LabDF.shape

(52619, 36)

In [5]:
import pandas as pd
VitalDF = pd.read_excel('/content/drive/MyDrive/ckdresearch2023/Assign/SummaryFile/WideTable/VitalDF.xlsx')
VitalDF.shape

(52619, 9)

In [6]:
VitalDF = VitalDF.iloc[:, 1:]
VitalDF

,PatientUID,PatientVisitUID,Slopes,CWhen,Systolic BP,Diastolic BP,BMI,BSA
0,2299,5812902,-7.158596,2561-12-12 09:33:19.420,138,76,20.72,1.38
1,2299,6947220,5.550691,2562-12-19 10:56:58.010,134,66,20.27,1.36
2,2299,8186767,-6.063710,2564-03-04 10:16:31.553,103,57,19.05,1.33
3,2299,9385133,-12.812245,NaN,128,62,NaN,NaN
4,2304,4916314,28.287500,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
52614,3319961,10553903,10.568365,2566-04-04 08:36:57.790,147,87,NaN,NaN
52615,3319971,10553916,-4.256753,2566-04-04 09:03:11.767,143,86,NaN,NaN
52616,3320048,10572126,-0.692078,2566-04-10 09:40:49.477,131,79,NaN,NaN
52617,3320050,10572565,7.217875,NaN,138,NaN,NaN,NaN


In [7]:
LabDF = LabDF.iloc[:, 1:]
LabDF

,PatientUID,PatientVisitUID,Slopes,L2:BUN,L3:Creatinine,L4:Sodium,L5:Potassium,L6:Chloride,L7:Bicarbonate,L8:Uric,...,L25:Parathyroid hormone,L26:Vitamin D,L27:UrineAlbumin/Cr ratio,L28:UrineProtein/Cr ratio,L29:UrineVolume (24hrs),L30:UrineUrea (24hrs),L31:UrineSodium (24hrs),L32:UrineProtein (24hrs),L1:eGFR,L14:WBC
0,2299,5812902,-7.158596,6.5,0.82,143,3.8,103,29.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.20,9.41
1,2299,6947220,5.550691,NaN,0.88,NaN,4.3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.02,NaN
2,2299,8186767,-6.063710,14.5,1.10,141,4.4,102,27.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.54,5.42
3,2299,9385133,-12.812245,15.4,0.72,131,4.2,96,28.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.47,NaN
4,2304,4916314,28.287500,20.8,1.08,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.10,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52614,3319961,10553903,10.568365,NaN,2.04,137,4.6,99,27.4,NaN,...,NaN,NaN,8.37,157.7,NaN,NaN,NaN,NaN,33.68,NaN
52615,3319971,10553916,-4.256753,28.3,1.87,141,5.2,104,23.5,NaN,...,NaN,NaN,13.37,71.1,NaN,NaN,NaN,NaN,26.47,NaN
52616,3320048,10572126,-0.692078,15.5,1.09,141,4.4,110,19.4,10.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.36,NaN
52617,3320050,10572565,7.217875,40.8,2.14,126,4.5,90,22.6,7.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.69,18.70


In [8]:
DiagDF = DiagDF.iloc[:, 1:]
DiagDF

,PatientUID,PatientVisitUID,Slopes,D1:Hypertension,D2:Diabetes millitus,D3:Dyslipidemia,D4:SLE,D5:Systemic sclerosis,D6:Rheumatoid arthritis,D7:HIV infection,D8:HBV infection,D9:HCV infection,D10:Ischemic heart disease,D11:Heart failure
0,2299,5812902,-7.158596,0,1,1,0,0,0,0,0,0,0,0
1,2299,6947220,5.550691,0,1,1,0,0,0,0,0,0,0,0
2,2299,8186767,-6.063710,1,1,1,0,0,0,0,0,0,0,0
3,2299,9385133,-12.812245,0,0,1,0,0,0,0,0,0,0,0
4,2304,4916314,28.287500,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52614,3319961,10553903,10.568365,1,0,0,0,0,0,0,0,0,0,0
52615,3319971,10553916,-4.256753,1,0,0,0,0,0,0,0,0,0,0
52616,3320048,10572126,-0.692078,0,0,0,2,0,0,0,0,0,0,0
52617,3320050,10572565,7.217875,0,0,0,0,0,0,0,0,0,0,0


In [9]:
DrugDF = DrugDF.iloc[:, 1:]
DrugDF

,PatientUID,PatientVisitUID,Slopes,G1M2:ARB,G3M1:Statin,G5M2:NSAIDs,G1M5:Diuretic,G1M7:Beta blocker,G1M3:Dihydropyridine CCB,G1M6:MRA,...,G1M9:Vasodilator,G1M8:Alpha blocker,G2M6:Insulin,G2M4:TZD,G4M2:Febuxostat,G3M2:Ezetimibe,G1M4:Non-dihydropyridine CCB,G5M1:Ketoanalog amino acid,G2M7:GLP1-RA,G4M3:Uricosuric agent
0,2299,5812902,-7.158596,100,100,34,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2299,6947220,5.550691,35,0,1,35,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2299,8186767,-6.063710,100,0,15,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2299,9385133,-12.812245,90,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2304,4916314,28.287500,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52614,3319961,10553903,10.568365,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52615,3319971,10553916,-4.256753,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52616,3320048,10572126,-0.692078,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52617,3320050,10572565,7.217875,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## **TestMerge**

In [10]:
import pandas as pd

# Assuming you have four dataframes: DrugDF1, DiagDF, DrugDF2, LabDF

# Merge DrugDF1, DiagDF, DrugDF2, and LabDF based on the common columns
merged_df = DrugDF.merge(DiagDF, on=['PatientUID', 'PatientVisitUID', 'Slopes'], how='outer') \
                    .merge(VitalDF, on=['PatientUID', 'PatientVisitUID', 'Slopes'], how='outer') \
                    .merge(LabDF, on=['PatientUID', 'PatientVisitUID', 'Slopes'], how='outer')

# The merged_df will contain all the data from the four dataframes, merged on the common columns.
merged_df

,PatientUID,PatientVisitUID,Slopes,G1M2:ARB,G3M1:Statin,G5M2:NSAIDs,G1M5:Diuretic,G1M7:Beta blocker,G1M3:Dihydropyridine CCB,G1M6:MRA,...,L25:Parathyroid hormone,L26:Vitamin D,L27:UrineAlbumin/Cr ratio,L28:UrineProtein/Cr ratio,L29:UrineVolume (24hrs),L30:UrineUrea (24hrs),L31:UrineSodium (24hrs),L32:UrineProtein (24hrs),L1:eGFR,L14:WBC
0,2299,5812902,-7.158596,100,100,34,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.20,9.41
1,2299,6947220,5.550691,35,0,1,35,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.02,NaN
2,2299,8186767,-6.063710,100,0,15,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.54,5.42
3,2299,9385133,-12.812245,90,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.47,NaN
4,2304,4916314,28.287500,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.10,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52614,3319961,10553903,10.568365,0,0,0,0,0,0,0,...,NaN,NaN,8.37,157.7,NaN,NaN,NaN,NaN,33.68,NaN
52615,3319971,10553916,-4.256753,0,0,0,0,0,0,0,...,NaN,NaN,13.37,71.1,NaN,NaN,NaN,NaN,26.47,NaN
52616,3320048,10572126,-0.692078,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.36,NaN
52617,3320050,10572565,7.217875,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.69,18.70


In [11]:
merged_df.shape

(52619, 75)

In [12]:
merged_df = merged_df.drop(columns='CWhen')

In [13]:
merged_df.shape

(52619, 74)

In [14]:
merged_df.to_excel('WideTable.xlsx')